In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

## Langsmith Tracking And Tracing
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")
os.environ["LANGCHAIN_TRACING_V2"]="true"

In [5]:
from langchain_groq import ChatGroq
model=ChatGroq(model="llama-3.3-70b-versatile")
response = model.invoke("what is Agentic AI?").content
print(response)

Agentic AI refers to artificial intelligence (AI) systems that are capable of autonomous decision-making, problem-solving, and action-taking, often with a sense of purpose, intention, or goal-oriented behavior. The term "agentic" comes from the word "agent," which in AI refers to a system that can perceive its environment, reason about it, and take actions to achieve its goals.

Agentic AI systems are designed to be proactive, adaptive, and responsive to changing circumstances, rather than simply reacting to pre-programmed rules or instructions. They can learn from experience, make decisions based on uncertain or incomplete information, and adjust their behavior accordingly.

Some key characteristics of agentic AI include:

1. **Autonomy**: Agentic AI systems can operate independently, making decisions and taking actions without human intervention.
2. **Goal-oriented behavior**: Agentic AI systems are designed to achieve specific goals or objectives, which can be defined by humans or l

In [6]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate([
    ("system", "You are an Ai Engineer, provide me the answer based on the question."),
    ("user", "{input}")
])

chain = prompt | model
chain

ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are an Ai Engineer, provide me the answer based on the question.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000019DE271EA50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019DE271F770>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [7]:
response = chain.invoke({"input": "tell me something about langserve?"})
print(response.content)

LangServe is an open-source language model serving system designed to simplify the deployment, management, and serving of large language models. It was developed by the team at Llama, a company that focuses on building AI models and tools.

LangServe aims to provide a robust and scalable solution for serving language models, allowing developers to easily integrate these models into their applications. Here are some key features of LangServe:

1. **Model Serving**: LangServe allows you to serve multiple language models simultaneously, making it easy to manage and switch between different models.
2. **Scalability**: LangServe is designed to handle large volumes of requests, making it suitable for applications with high traffic.
3. **Flexibility**: LangServe supports a wide range of language models, including popular models like BERT, RoBERTa, and XLNet.
4. **Low-Latency**: LangServe is optimized for low-latency responses, ensuring that your application can respond quickly to user request

### Output parsers

In [9]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain1 = prompt | model | output_parser

response = chain1.invoke({"input": "tell me something about langserve?"})
print(response)

LangServe is an open-source, language-agnostic, and framework-agnostic API gateway for large language models (LLMs) and other AI/ML models. It provides a unified interface for deploying, managing, and serving machine learning models in a scalable and secure manner.

Here are some key features and benefits of LangServe:

**Key Features:**

1. **Model Serving**: LangServe allows you to deploy and serve multiple machine learning models, including LLMs, computer vision models, and more.
2. **API Gateway**: It provides a unified API interface for clients to interact with the models, abstracting away the underlying model implementation details.
3. **Model Management**: LangServe offers features for model versioning, monitoring, and updating, making it easier to manage the lifecycle of machine learning models.
4. **Scalability**: It is designed to scale horizontally, allowing you to handle large volumes of requests and traffic.
5. **Security**: LangServe provides features for authentication, 

In [13]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

output_parser_json = JsonOutputParser()

prompt_json = PromptTemplate(
    template="Answer the user query in \n {format_instruction} \n {query}",
    input_variables = ['query'],
    partial_variables = {"format_instruction": output_parser_json.get_format_instructions()}
)

prompt_json

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'Return a JSON object.'}, template='Answer the user query in \n {format_instruction} \n {query}')

In [15]:
chain_json = prompt_json | model | output_parser_json
response = chain_json.invoke({"query": "tell me something about langserve?"})
print(response)

{'name': 'LangServe', 'description': 'LangServe is a language server that provides a common interface for various programming languages, allowing for features like code completion, debugging, and diagnostics.', 'features': ['Code completion', 'Debugging', 'Diagnostics', 'Code refactoring', 'Code navigation'], 'benefits': ['Improved coding efficiency', 'Enhanced code quality', 'Better error detection and correction', 'Simplified code maintenance and refactoring'], 'supported_languages': ['C', 'C++', 'Java', 'Python', 'JavaScript', 'Ruby', 'Swift', 'Go', 'Rust'], 'supported_editors': ['Visual Studio Code', 'Sublime Text', 'Atom', 'Brackets', 'Emacs']}


### With Pydantic

In [22]:
from pydantic import BaseModel,Field

class WeatherResponse(BaseModel):
    location: str
    temperature: float
    condition: str

output_parser_pydantic = JsonOutputParser(pydantic_object=WeatherResponse)

prompt_weather = PromptTemplate(
    template="Answer the user query in \n {format_instruction} \n {query}",
    input_variables = ['query'],
    partial_variables = {"format_instruction": output_parser_pydantic.get_format_instructions()}
)

prompt_weather

PromptTemplate(input_variables=['query'], input_types={}, partial_variables={'format_instruction': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"location": {"title": "Location", "type": "string"}, "temperature": {"title": "Temperature", "type": "number"}, "condition": {"title": "Condition", "type": "string"}}, "required": ["location", "temperature", "condition"]}\n```'}, template='Answer the user query in \n {format_instruction} \n {query}')

In [23]:
chain_weather = prompt_weather | model | output_parser_pydantic

response = chain_weather.invoke({"query": "what the weather in chennai today"})
print(response)

{'location': 'Chennai', 'temperature': 32.5, 'condition': 'Sunny'}
